# Data Based Problems

The DESDEO framework provides handling of data-driven optimization problems. Some methods, such as E-NAUTILUS in `desdeo-mcdm`, find the most preffered solution from a provided dataset. Other methods, such as most of the EA's from `desdeo-emo`, require a surrogate model to be trained for each of the objectives. The `desdeo_problem` provides support for both of these cases. 

For data based problems, use the data specific objective/problem classes

In [1]:
import pandas as pd
import numpy as np

VectorDataObjective is an objective class that can handle data, as well as multi-objective evaluators.

The GaussianProcessRegressor here is same as the one in scikit-learn with one small difference. The predict method has been replaced to return uncertainity values (in the form of standard deviation of the prediction) by default. It supports hyperparameters in the same format as the sklearn method.

In [2]:
from desdeo_problem.Objective import VectorDataObjective as VDO
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

### Creating some random data

'a' and 'b' are randomly generated between 0 and 1.

f1 = a + b  
f2 = a * b

For data-driven problems, make sure that the input dataset is in the pandas DataFrame format, with the column names being the same as the variable/objective names.

In [3]:
data = np.random.rand(100,2)

f1 = (data[:,0]+data[:,1]).reshape(-1,1)
f2 = (data[:,0]*data[:,1]).reshape(-1,1)

data = np.hstack((data, f1, f2))

X = ['a','b']
y = ['f1','f2']
datapd = pd.DataFrame(data, columns=X+y)
datapd.head()

,a,b,f1,f2
0,0.866445,0.492558,1.359003,0.426775
1,0.848106,0.476255,1.324360,0.403914
2,0.290107,0.533858,0.823966,0.154876
3,0.261330,0.317741,0.579072,0.083035
4,0.303202,0.813731,1.116932,0.246724


### Using VectorDataObjective class

The `VectorDataObjective` class takes as its input the data in a dataframe format and the objective names in a list.

In [4]:
obj = VDO(data=datapd, name=y)

### Training surrogate models

Pass the surrogate modelling technique and the model parameters to the train method of the objective instance.

If only one modelling technique is passed, the `model_parameters` should be a dict (or None) and this will be used for all the objectives.

If multiple modelling techniques are passed, `models` should be the list of modelling techniques, and `model_parameters` should be a list of dicts. The length of these lists should be the same as the number of objectives and each list element will be used to train one objective in order.

In [5]:
obj.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})

### Using surrogate models to evaluate objective values

Use the obj.evaluate method to get predictions. Note that `use_surrogates` should be true.

In [6]:
print(obj.evaluate(np.asarray([[0.5,0.3]]), use_surrogate=True))

Objective Evaluation Results Object 
Objective values are: 
         f1        f2
0  0.799997  0.150003
Uncertainity values are: 
         f1       f2
0  0.000363  0.00056



In [7]:
obj._model_trained

{'f1': True, 'f2': True}

### Creating data problem class

Creating the objective class should be bypassed for now, use `DataProblem` class directly with the data in a dataframe.

The `DataProblem` provides a `train` method which trains all the objectives sequentially. The input arguments for this train method is the same as that of the `VectorDataObjective` class.

To make sure that the `evaluate` method uses the surrogate models for evaluations, pass the `use_surrogate=True` argument.

In [8]:
from desdeo_problem.Problem import DataProblem

In [9]:
maximize = pd.DataFrame([[True, False]], columns=['f1','f2'])
prob = DataProblem(data=datapd, objective_names=y, variable_names=X, maximize=maximize)

In [10]:
prob.train(GaussianProcessRegressor)

In [11]:
print(prob.evaluate(np.asarray([[0.1,0.8], [0.5,0.3]]), use_surrogate=True))

Evaluation Results Object 
Objective values are: 
[[0.89999818 0.07999432]
 [0.80000079 0.15000032]]
Constraint violation values are: 
None
Fitness values are: 
[[-0.89999818  0.07999432]
 [-0.80000079  0.15000032]]
Uncertainity values are: 
[[0. 0.]
 [0. 0.]]



/home/rishi/.virtualenvs/desdeo-problem/lib/python3.7/site-packages/sklearn/gaussian_process/gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
/home/rishi/.virtualenvs/desdeo-problem/lib/python3.7/site-packages/sklearn/gaussian_process/gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


## Lipschitian models

In [12]:
from desdeo_problem.surrogatemodels.lipschitzian import LipschitzianRegressor

In [13]:
prob = DataProblem(data=datapd, objective_names=y, variable_names=X)

In [14]:
prob.train(LipschitzianRegressor)

In [15]:
print(prob.evaluate(np.asarray([[0.1,0.8], [0.5,0.3]]), use_surrogate=True))

Evaluation Results Object 
Objective values are: 
[[0.9        0.0654458 ]
 [0.8        0.13795083]]
Constraint violation values are: 
None
Fitness values are: 
[[0.9        0.0654458 ]
 [0.8        0.13795083]]
Uncertainity values are: 
[[1.88737914e-15 3.16508168e-02]
 [6.66133815e-16 3.69551016e-02]]

